## v1
Ver MILE

**Creacion de los atributos, labels o etiquetas de clasificacion de cada imagen**

In [0]:
label_names = sorted(item.name for item in data_root.glob('*/') if item.is_dir())
label_to_index = dict((name, index) for index,name in enumerate(label_names))
all_image_labels = [label_to_index[pathlib.Path(path).parent.name] for path in all_image_paths]

**Creacion del dataset que inyecta los datos al modelo**

In [0]:
tfds = tf.data.Dataset.from_tensor_slices((all_image_paths, all_image_labels))

**Pre-procesar la imagen convirtiendola a algo que el modelo pueda entender**

In [0]:
def load_and_preprocess_from_path_label(path, label):
  #def _input_fn():
    def _preprocess_image(image):
      image = tf.image.decode_jpeg(image, channels=3)
      image = tf.image.resize(image, IMAGE_SIZE)
      image /= 255.9 # se esta normalizando a [0, 1]
      return image
      
    image_encoded = tf.read_file(path)
    image_decoded = _preprocess_image(image_encoded)
     
    return image_decoded, label
  #return _input_fn

In [0]:
image_label_ds = tfds.map(load_and_preprocess_from_path_label)
image_label_ds

<DatasetV1Adapter shapes: ((224, 224, 3), ()), types: (tf.float32, tf.int32)>

In [0]:
# Setting a shuffle buffer size as large as the dataset ensures that the data is
# completely shuffled.
ds = image_label_ds.shuffle(buffer_size=image_count)
ds = ds.repeat()
ds = ds.batch(BATCH_SIZE)
# `prefetch` lets the dataset fetch batches, in the background while the model is training.
ds = ds.prefetch(buffer_size=AUTOTUNE)
ds

In [0]:
ds = image_label_ds.apply(tf.data.experimental.shuffle_and_repeat(buffer_size=image_count))
ds = ds.batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)
ds

Dataset

In [0]:
def read_dataset(filename, mode, batch_size):
  
  #Creacion de la ruta de las imagenes
  data_root = pathlib.Path(filename)
  all_image_paths = list(data_root.glob('*/*'))
  all_image_paths = [str(path) for path in all_image_paths]
  random.shuffle(all_image_paths)
  
  label_names = sorted(item.name for item in data_root.glob('*/') if item.is_dir())
  label_to_index = dict((name, index) for index,name in enumerate(label_names))
  all_image_labels = [label_to_index[pathlib.Path(path).parent.name]
                    for path in all_image_paths]
  
  
  
  #Decodificar y cambiar el size de la imagen
  def _load_and_preprocess_image(path):
  #def _input_fn():
    def _preprocess_image(image):
      image = tf.image.decode_jpeg(image, channels=3)
      image = tf.image.resize(image, IMAGE_SIZE)
      image /= 255.9 # se esta normalizando a [0, 1]
      return image

    image_encoded = tf.read_file(path)
    image_decoded = _preprocess_image(image_encoded)

    return image_decoded
  #return _input_fn
  
  #Toma la ruta de las imagenes, la decodifica, crea el tensor de atributos
  paths_ds = tf.data.Dataset.from_tensor_slices(all_image_paths) 
  image_ds = paths_ds.map(_load_and_preprocess_image)
  
  # se crea un TFRecord para mejorar la eficiencia
  ds = image_ds.map(tf.serialize_tensor)
  tfrec = tf.data.experimental.TFRecordWriter('images.tfrec')
  tfrec.write(ds)
  
  ## With the preprocessing cached, data can be loaded from the TFrecord file quite efficiently. 
  ##Just remember to de-serialized tensor before trying to use it.
  ds = tf.data.TFRecordDataset('images.tfrec')
  def parse(x):
    result = tf.parse_tensor(x, out_type=tf.float32)
    result = tf.reshape(result, [*IMAGE_SIZE, 3])
    return result
  ds = ds.map(parse, num_parallel_calls=AUTOTUNE)
  
  label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(all_image_labels, tf.int64))
  image_count = len(all_image_paths)
  
  #Se ingest el dataset
  ds = tf.data.Dataset.zip((ds, label_ds))
  
  if mode == tf.estimator.ModeKeys.TRAIN:
    num_epochs = None #Loop indefinidamente
    ds = ds.shuffle(buffer_size = 10 * batch_size) # video 7:10 https://www.youtube.com/watch?v=uIcqeP7MFH0
    ds = ds.apply(tf.data.experimental.shuffle_and_repeat(buffer_size=10*batch_size)) #image_count
  else:
    num_epochs = 1 #end-of-input after this
  
  ds = ds.repeat(epochs).batch(BATCH_SIZE).prefetch(AUTOTUNE) # video 7:48
  return ds.make_one_shot_iterator().get_next()

# Predicciones

In [0]:
AUTO = tf.data.experimental.AUTOTUNE
from matplotlib import pyplot as plt
CLASSES = ['dogs', 'cats'] # do not change, maps to the labels in the data (folder names)

def dataset_to_numpy_util(dataset, N):
  dataset = dataset.batch(N)
  #from IPython.core.debugger import Tracer; Tracer()() 
  if tf.executing_eagerly():
    # In eager mode, iterate in the Datset directly.
    for images, labels in dataset:
      numpy_images = images.numpy()
      numpy_labels = labels.numpy()
      break;
      
  else: # In non-eager mode, must get the TF note that 
        # yields the nextitem and run it in a tf.Session.
    get_next_item = dataset.make_one_shot_iterator().get_next()
    with tf.Session() as ses:
      numpy_images, numpy_labels = ses.run(get_next_item)

  return numpy_images, numpy_labels

def read_tfrecord(example):
    features = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "class": tf.io.FixedLenFeature([], tf.int64),  # shape [] means scalar
    }
    example = tf.parse_single_example(example, features)
    image = tf.image.decode_jpeg(example['image'], channels=3)
    image = tf.cast(image, tf.float32) / 255.0  # convert image to floats in [0, 1] range
    image = tf.reshape(image, [229, 229, 3]) # explicit size will be needed for TPU
    class_label = example['class']
    return image, class_label


def load_dataset(filenames):
  # read from TFRecords. For optimal performance, use "interleave(tf.data.TFRecordDataset, ...)"
  # to read from multiple TFRecord files at once and set the option experimental_deterministic = False
  # to allow order-altering optimizations.

  option_no_order = tf.data.Options()
  option_no_order.experimental_deterministic = False

  dataset = tf.data.Dataset.list_files(filenames)
  dataset = dataset.with_options(option_no_order)
  #dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=16)
  dataset = dataset.interleave(tf.data.TFRecordDataset, cycle_length=16, num_parallel_calls=AUTO) # faster
  dataset = dataset.map(read_tfrecord, num_parallel_calls=AUTO)
  return dataset

def display_9_images_from_dataset(dataset):
  subplot=331
  plt.figure(figsize=(13,13))
  images, labels = dataset_to_numpy_util(dataset, 9)
  for i, image in enumerate(images):
    title = CLASSES[labels[i]]
    subplot = display_one_flower(image, title, subplot)
    if i >= 8:
      break;
              
  plt.tight_layout()
  plt.subplots_adjust(wspace=0.1, hspace=0.1)
  plt.show()

  
#display_9_images_from_dataset(load_dataset(pred_files))
#dataset_to_numpy_util(load_dataset(pred_files),9)
for images, labels in load_dataset('/tmp/datasets/dogscats/sample/train/cats/cat.4600.jpg'):
  numpy_images = images.numpy()
  numpy_labels = labels.numpy()